# Data Wrangling

https://www.cbp.gov/newsroom/stats

https://www.ice.gov/detain/detention-management

https://www.uscis.gov/tools/reports-and-studies/immigration-and-citizenship-data

https://ohss.dhs.gov/topics/immigration#other-resources 


# Ideas
- Write about H1B, H2A, H2B, or other visas
- Write about detained individual counts at the border
- Write about detained individuals within the US
- Forecast immigration data
- Forecast impacts of policy changes

- Combine company visa information with their stock information
- Investigate if there are any correlations between stock information and visa information
- Can we make price movement predictions with visa information of publicly listed companies



In [94]:
# Imports
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from yfinance import ticker
import json
from io import StringIO
import re

In [95]:
# Gets a list of all listed companies from SEC's EDGAR database
#"https://www.sec.gov/files/company_tickers.json"

with open('data/company_tickers.json', 'r') as f:
    data = json.load(f)
# companies = [entry["ticker"] for entry in response.values()]
# print(companies[:10])  # Print first 10 tickers

In [96]:
### More robust data wrangling pipeline once we get remove_meta_characters optimized / finish debugging files ###

def read_tsv(file_path):
    """Reads a TSV file with UTF-8 encoding."""
    with open(file_path, "r", encoding="utf-8") as f:
        return pd.read_csv(StringIO(f.read()), delimiter="\t")
    
def remove_meta_characters(input_string):
    input_string = str(input_string)
    input_string = re.sub(r'\s+', ' ', input_string).strip() 
    input_string = re.sub(r'\b(INCORPORATED|INC|CORPORATION|CORP|NA|N A|LTD|LIMITED|LLC|LLP|PLC|GROUP|GRP|GR|HOLDINGS|COMPANY|CO|LP|PARTNERSHIP)\b',
                          '', str(input_string).upper())
    input_string = re.sub(r'/[^/]+/', ' ', input_string)
    input_string = input_string.replace('.', '').replace(',', '').replace('&', 'AND')
    input_string = re.sub(r'\s+', ' ', input_string).strip() 
    return input_string

titles = [company['title'].upper() for company in data.values()]
standard_titles = [remove_meta_characters(title) for title in titles]

visa_files = {"h1b": ['data/h1b_2016_2009.csv', 'data/h1b_2024_2017.csv'],
              "h2a": ['data/h2a_2019_2015.csv', 'data/h2a_2024_2020.csv'],
              "h2b": ['data/h2b_2019_2015.csv', 'data/h2b_2025_2020.csv']}

df_visas = []
for visa_type, files in visa_files.items():
    df_list = [read_tsv(file) for file in files]
    df = pd.concat(df_list, axis=0)
    df["employer"] = df["Employer (Petitioner) Name"].str.upper().apply(remove_meta_characters)
    df_filtered = df[df["employer"].isin(standard_titles)].copy()
    df_filtered.loc[:, "type"] = visa_type
    df_visas.append(df_filtered)
 
df_visas_combined = pd.concat(df_visas)

drop_cols = ['Line by line', 'Employer (Petitioner) Name', 'Tax ID', 'Initial Denial', 'Continuing Denial', 'Index()', 'Consular_processed', 'New Employment Denial', 'Continuation Denial', 'Change with Same Employer Denial', 'New Concurrent Denial', 'Change of Employer Denial', 'Amended Denial', 'Consular_Processed', 'ETA Case Number']
df_visas_clean = df_visas_combined.drop(drop_cols, axis = 1)

df_visas_clean.shape


C:\Users\peter\AppData\Local\Temp\ipykernel_22096\2973020064.py:6: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")
C:\Users\peter\AppData\Local\Temp\ipykernel_22096\2973020064.py:6: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")
C:\Users\peter\AppData\Local\Temp\ipykernel_22096\2973020064.py:6: DtypeWarning: Columns (4,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")
C:\Users\peter\AppData\Local\Temp\ipykernel_22096\2973020064.py:6: DtypeWarning: Columns (16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")


(19349, 24)

In [97]:

h1b_2009_path = 'data/h1b_2016_2009.csv'
h1b_2017_path = 'data/h1b_2024_2017.csv'

h2a_2015_path = 'data/h2a_2019_2015.csv'
h2a_2020_path = 'data/h2a_2024_2020.csv'

h2b_2015_path = 'data/h2b_2019_2015.csv'
h2b_2020_path = 'data/h2b_2025_2020.csv'

#Had to convert all encoding to UTF8 via notepad++
def read_tsv(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return pd.read_csv(StringIO(f.read()), delimiter="\t")

# Read all files using the function
df_h1b_a = read_tsv(h1b_2009_path)
df_h1b_b = read_tsv(h1b_2017_path)

df_h2a_a = read_tsv(h2a_2015_path)
df_h2a_b = read_tsv(h2a_2020_path)

df_h2b_a = read_tsv(h2b_2015_path)
df_h2b_b = read_tsv(h2b_2020_path)

C:\Users\peter\AppData\Local\Temp\ipykernel_22096\3014528854.py:13: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")
C:\Users\peter\AppData\Local\Temp\ipykernel_22096\3014528854.py:13: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")
C:\Users\peter\AppData\Local\Temp\ipykernel_22096\3014528854.py:13: DtypeWarning: Columns (4,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")
C:\Users\peter\AppData\Local\Temp\ipykernel_22096\3014528854.py:13: DtypeWarning: Columns (16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")


In [98]:
df_h1b = pd.concat([df_h1b_a, df_h1b_b], axis=0)

df_h2a = pd.concat([df_h2a_a, df_h2a_b], axis=0)

df_h2b = pd.concat([df_h2b_a, df_h2b_b], axis=0)


In [99]:
print(df_h1b.columns)
print(df_h2a.columns)
print(df_h2b.columns)

Index(['Line by line', 'Fiscal Year   ', 'Employer (Petitioner) Name',
       'Tax ID', 'Industry (NAICS) Code', 'Petitioner City',
       'Petitioner State', 'Petitioner Zip Code', 'Initial Approval',
       'Initial Denial', 'Continuing Approval', 'Continuing Denial'],
      dtype='object')
Index(['Index()', 'Action Fiscal Year', 'Employer (Petitioner) Name', 'Tax ID',
       'Industry', 'Occupation (SOC) Code', 'Petitioner City',
       'Petitioner State', 'Petitioner Zip Code', 'Worksite State',
       'Consular_processed', 'Wage Rate Band', 'New Employment Approval',
       'New Employment Denial', 'Continuation Approval', 'Continuation Denial',
       'Change with Same Employer Approval',
       'Change with Same Employer Denial', 'New Concurrent Approval',
       'New Concurrent Denial', 'Change of Employer Approval',
       'Change of Employer Denial', 'Amended Approval', 'Amended Denial'],
      dtype='object')
Index(['Index()', 'Cap Fiscal Year', 'Cap Type', 'Employer (Petiti

In [100]:
def remove_meta_characters(input_string):
    input_string = str(input_string)
    input_string = re.sub(r'\s+', ' ', input_string).strip() 
    input_string = re.sub(r'\b(INCORPORATED|INC|CORPORATION|CORP|NA|N A|LTD|LIMITED|LLC|LLP|PLC|GROUP|GRP|GR|HOLDINGS|COMPANY|CO|LP|PARTNERSHIP)\b',
                          '', str(input_string).upper())
    input_string = re.sub(r'/[^/]+/', ' ', input_string)
    input_string = input_string.replace('.', '').replace(',', '').replace('&', 'AND')
    input_string = re.sub(r'\s+', ' ', input_string).strip() 
    return input_string

In [101]:
titles = [company['title'].upper() for company in data.values()]

standard_titles = [remove_meta_characters(title) for title in titles]

In [102]:
df_h1b['employer'] = df_h1b['Employer (Petitioner) Name'].str.upper().apply(remove_meta_characters)
df_h2a['employer'] = df_h2a['Employer (Petitioner) Name'].str.upper().apply(remove_meta_characters)
df_h2b['employer'] = df_h2b['Employer (Petitioner) Name'].str.upper().apply(remove_meta_characters)

In [103]:
print('H1B Crossover Count: ' + str(sum(df_h1b["employer"].isin(standard_titles))))
print('H2A Crossover Count: ' + str(sum(df_h2a["employer"].isin(standard_titles))))
print('H2B Crossover Count: ' + str(sum(df_h2b["employer"].isin(standard_titles))))


H1B Crossover Count: 19226
H2A Crossover Count: 73
H2B Crossover Count: 50


In [105]:
df_h1b_listed = df_h1b[ df_h1b["employer"].isin(standard_titles) ].copy()
df_h1b_listed.loc[:, "type"] = 'h1b'

df_h2a_listed = df_h2a[ df_h2a["employer"].isin(standard_titles) ].copy()
df_h2a_listed.loc[:, "type"] = 'h2a'

df_h2b_listed = df_h2b[ df_h2b["employer"].isin(standard_titles) ].copy()
df_h2b_listed.loc[:, "type"] = 'h2b'



In [146]:
print(df_h1b.columns)
print(df_h2a.columns)
print(df_h2b.columns)

Index(['Line by line', 'Fiscal Year   ', 'Employer (Petitioner) Name',
       'Tax ID', 'Industry (NAICS) Code', 'Petitioner City',
       'Petitioner State', 'Petitioner Zip Code', 'Initial Approval',
       'Initial Denial', 'Continuing Approval', 'Continuing Denial',
       'employer'],
      dtype='object')
Index(['Index()', 'Action Fiscal Year', 'Employer (Petitioner) Name', 'Tax ID',
       'Industry', 'Occupation (SOC) Code', 'Petitioner City',
       'Petitioner State', 'Petitioner Zip Code', 'Worksite State',
       'Consular_processed', 'Wage Rate Band', 'New Employment Approval',
       'New Employment Denial', 'Continuation Approval', 'Continuation Denial',
       'Change with Same Employer Approval',
       'Change with Same Employer Denial', 'New Concurrent Approval',
       'New Concurrent Denial', 'Change of Employer Approval',
       'Change of Employer Denial', 'Amended Approval', 'Amended Denial',
       'employer'],
      dtype='object')
Index(['Index()', 'Cap Fisca

In [147]:
df_visas = pd.concat([df_h1b_listed, df_h2a_listed, df_h2b_listed])
df_visas.shape

(13610, 39)

h1b_drop_cols = ['Line by line', 'Employer (Petitioner) Name', 'Tax ID', 'Initial Denial', 'Continuing Denial', 'Initial Approval', 'Continuing Approval']
#Adding the columns does not work since they are currently strings, to_numeric not working due to commas
df_h1b_listed['tot'] = df_h1b_listed['Initial Approval'] + df_h1b_listed['Continuing Approval']
df_h1b_clean = df_h1b_listed.drop(h1b_drop_cols, axis = 1)
df_h1b_clean

In [18]:
df_visas.columns


Index(['Line by line', 'Fiscal Year   ', 'Employer (Petitioner) Name',
       'Tax ID', 'Industry (NAICS) Code', 'Petitioner City',
       'Petitioner State', 'Petitioner Zip Code', 'Initial Approval',
       'Initial Denial', 'Continuing Approval', 'Continuing Denial',
       'employer', 'type', 'Index()', 'Action Fiscal Year', 'Industry',
       'Occupation (SOC) Code', 'Worksite State', 'Consular_processed',
       'Wage Rate Band', 'New Employment Approval', 'New Employment Denial',
       'Continuation Approval', 'Continuation Denial',
       'Change with Same Employer Approval',
       'Change with Same Employer Denial', 'New Concurrent Approval',
       'New Concurrent Denial', 'Change of Employer Approval',
       'Change of Employer Denial', 'Amended Approval', 'Amended Denial',
       'Cap Fiscal Year', 'Cap Type', 'Work Site State', 'Consular_Processed',
       'Hourly Wage', 'ETA Case Number'],
      dtype='object')

In [127]:
drop_cols = ['Line by line', 'Employer (Petitioner) Name', 'Tax ID', 'Initial Denial', 'Continuing Denial', 'Index()', 'Consular_processed', 'New Employment Denial', 'Continuation Denial', 'Change with Same Employer Denial', 'New Concurrent Denial', 'Change of Employer Denial', 'Amended Denial', 'Consular_Processed', 'ETA Case Number']
df_visas_clean = df_visas.drop(drop_cols, axis = 1)

In [106]:
df_visas_clean.head()

,Fiscal Year,Industry (NAICS) Code,Petitioner City,Petitioner State,Petitioner Zip Code,Initial Approval,Continuing Approval,employer,type,Action Fiscal Year,...,New Employment Approval,Continuation Approval,Change with Same Employer Approval,New Concurrent Approval,Change of Employer Approval,Amended Approval,Cap Fiscal Year,Cap Type,Work Site State,Hourly Wage
63,2016.0,31-33 - Manufacturing,ROCK HILL,SC,29730.0,0,7,3D SYSTEMS,h1b,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,2016.0,31-33 - Manufacturing,SAINT PAUL,MN,55144.0,4,13,3M,h1b,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,2016.0,31-33 - Manufacturing,ST PAUL,MN,55144.0,2,8,3M,h1b,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,2016.0,NaN,SAN JOSE,CA,95131.0,12,8,8X8,h1b,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,2016.0,"54 - Professional, Scientific, and Technical S...",SAN JOSE,CA,95131.0,0,1,8X8,h1b,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
import time
def fetch_stock_data(tickers):
    all_data = {}
    print('Downloading yfinance historical price data 2000/01/01 to 2025/03/16')
    num_downloaded = 0
    for ticker in tickers:
        try:
            data = yf.download(ticker, start="2000-01-01", end="2025-03-16", progress = False)
            all_data[ticker] = data
            num_downloaded +=1
            time.sleep(2.2)  # Sleep for 2 seconds between requests to avoid being blocked
        except Exception as e:
            print(f"Error downloading {ticker}: {e}")
            time.sleep(2.2)

    print('Successfully downloaded ' + str(num_downloaded/len(tickers)))
    #Save a csv for offline work
    sp500_data = pd.concat(all_data, axis=1)
    sp500_data.to_csv("sp500_data.csv")
    
    print("S&P 500 historical data saved to sp500_data.csv")
    return sp500_data

#Only run this once on your system
#Make sure NOT to track the sp500_data csv
#sp500csv made for offline work
#fetch_stock_data